<a href="https://colab.research.google.com/github/csgm2328/Emily/blob/master/Stock_LSTM_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as pdr #data 받아오는 api 라이브러리
from pandas import json_normalize
import numpy as np
import plotly as py
import plotly.graph_objs as go
import requests
from datetime import datetime
import traceback
import os
from keras import metrics
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from pandas.util.testing import assert_frame_equal
from sklearn.preprocessing import MinMaxScaler

plt.style.use('bmh')

#Request URL API

In [ ]:
stock_url = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
stock_url.종목코드 = stock_url.종목코드.map('{:06d}'.format)
stock_url = stock_url[['회사명', '종목코드']] 
stock_url = stock_url.rename(columns={'회사명':'name', '종목코드':'code'})

In [ ]:
item_name = '' # 원하는 종목 이름 입력
code = stock_url.query("name=='{}'".format(item_name))['code'].to_string(index=False).strip() #strip 앞뒤공백제거
#DataReader용
#dr_code = code + '.KS' 
dr_code = code + '.KQ'
print(item_name, dr_code)

#DataReader crawling

In [ ]:
#URL 없이 상당히 쉬운 DataReader를 이용한 크롤링
start = (2012, 1, 1)
start_date = datetime(*start)
end = datetime.today()
df = pdr.DataReader(dr_code, 'yahoo', start_date, end) #031430.KS 신세계인터내셔날, 코스닥 데이터는 왜 오늘치밖에 안나오냐?
df.astype(np.int64)

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/KOSDAQ200

In [ ]:
row = pd.DataFrame([['날짜',	'Close',	'전일비',	'시가',	'고가',	'저가',	'거래량']])
c = [['날짜',	'Close',	'전일비',	'시가',	'고가',	'저가',	'거래량']]
x = np.asarray(row)
y = np.asarray(df)
x = np.vstack((x,y))
x.shape

In [ ]:
df = pd.read_csv('240810 원익IPS.csv')
scaler = MinMaxScaler()
real_price = df['32300.0'].tolist()
price = scaler.fit_transform(df[['32300.0']]).reshape(-1).tolist()
print('오늘은',datetime.today(), real_price[-1], len(real_price), '개')

#Args

In [ ]:
from keras import optimizers
#args
n_Memcell = 128
window_size = 100 #여기도 60win에서 두번 실패
b_state = True
predict_day = 10
#return_sequences = 여러 층일때, 각 시퀀스에서 출력할지 말지 결정. many to many 일때

n_batch = 1
n_epoch =  1000#colab 12시간 제한 특성 때문
n_drop = 0.2
learning_rate = 0.001 #default
ac_func = 'relu'
loss_func = 'mse'
optim_func = optimizers.Adam(lr=learning_rate)
#decay_rate = 0.001 #신경망에서는 수학적으로 동일한 L2 놈을 대신사용한다.
#optim_func = optimizers.Adadelta(learning_rate = learning_rate) #0.1, 0.001 둘다 과소적합
item_name = '원익IPS'
start = (2012, 1, 1)
start_date = datetime(*start)
args_str = '['+str(datetime.date(start_date)) + ' ~ ' + str(datetime.date(datetime.today())) + '] ' + str(n_Memcell) + 'cell '+ str(window_size) +'win '+ str(n_batch) +'bat ' + str(n_epoch) + 'ep '+ str(n_drop) + 'drop ' + str(b_state) + ' ' + str(learning_rate) + 'lr'
Title = item_name + ': ' + args_str
Title

#make DataSet

In [ ]:
x = []  #5일의 데이터를 넣고 다음날을 예측하는 방식 (특징벡터 - 훈련집합)
y = []  #5+1일의 데이터를 넣고 (결과값 - 훈련집합)

for i in range(len(price) - window_size):   #1916-5 = 1911 Range는 마지막 1910까지임
    x.append([price[i+j] for j in range(window_size)])
    y.append(price[window_size + i])
    
#print(x[-1])  # [1911,5] list
#print(price[-1]) #[1916, 1]
#print(y[-1])  #[1911, 1]

In [ ]:
x = np.asarray(x)
y = np.asarray(y)
print(x.shape) # [1911,5] list
print(type(y))

In [ ]:
train_test_split = int(len(x)*0.3) #1200개에서 1357개로 증가후에 acc: 0.0017 --> 0.0015
split_pt = train_test_split + window_size

x_test = x[:train_test_split, :]
y_test = y[:train_test_split]

x_train = x[train_test_split:, :]
y_train = y[train_test_split:]

xtrain = x_train.reshape((x_train.shape[0], x_train.shape[1], 1)) # 2차원 데이터 3차원화, LSTM의 데이터 생성할때 주의해야하는 포인트, [sequence,  batch, input_dim]
xtest =  np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  #test 집합을 늘리면 미래를 예측할 수 있을까?

print(xtrain.shape)
print(x_test.shape) 

In [ ]:
#history Metrics
import keras
class CustomHistory(keras.callbacks.Callback):
  def init(self):
    self.losses = []
    self.accs = []
  def on_epoch_end(self, batch, logs={}):
    self.losses.append(logs.get('loss'))
    self.accs.append(logs.get('acc'))   

In [ ]:
#Model Define
model = Sequential()
model.add(LSTM(n_Memcell, batch_input_shape =(n_batch, window_size, 1), stateful=b_state))
model.add(Dropout(n_drop))
#model.add(Activation(ac_func))
model.add(Dense(1))

model.compile(loss=loss_func, optimizer=optim_func, metrics=['mae', 'acc'])
model.summary()

In [ ]:
%%time
report = CustomHistory()
report.init()
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0.0001, patience=10, mode='auto') 

for i in range(n_epoch):
    print('[%d] ' %i)
    model.fit(xtrain, y_train, epochs=1, batch_size=n_batch, callbacks=[report, early_stopping],verbose=2)
    model.reset_states()



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/My Drive/Colab Notebooks/LSTM_models

In [ ]:
from keras.models import load_model
model.save(Title + '.h5')

In [ ]:
trainScore = model.evaluate(xtrain, y_train, batch_size=n_batch, verbose=0)
testScore = model.evaluate(xtest, y_test, batch_size=n_batch, verbose=0)
print("mae: (%.5f, %.5f)" %(trainScore[1], testScore[1]))

In [ ]:
print(trainScore)
testScore

In [ ]:
loss_ax = plt.subplot()
acc_ax = loss_ax.twinx()

loss_ax.plot(report.losses, 'y', label='train_loss')
acc_ax.plot(report.accs, 'b', label='train_acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuracy')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
#train, test prediction
train_pred = model.predict(xtrain,batch_size=n_batch) # x[0~window_size-1] 만큼의 특징벡터를 학습해서 학습데이터 수 
#model.reset_states()
test_pred = model.predict(xtest,batch_size=n_batch)
#model.reset_states()
real_test_pred = scaler.inverse_transform(test_pred)

In [ ]:
tom_real = xtrain[-1]
tom_real = np.vstack((tom_real[1:], price[-1])) #test_pred[-1] 대신에 price[-1]
tom_real.shape

tom_pred = xtest[-1]
tom_pred = np.vstack((tom_pred[1:], test_pred[-1])) #마지막 데이터에 예측값
tom_pred.shape

In [ ]:
#학습된 모델로 예측하기
seq_in = tom_real #오늘꺼 예측자를 마지막 데이터로 추가해준 input 샘플 or real 데이터 추가
after_pred = np.zeros((predict_day,1))

for i in range(predict_day):
  sample_in = seq_in.reshape(1,window_size,1)
  after = model.predict(sample_in, batch_size=n_batch)
  model.reset_states()
  seq_in = np.vstack((seq_in[1:], after))
  after_pred[i] = after
  
#model.reset_states()
real_after_pred = scaler.inverse_transform(after_pred)

In [ ]:
#학습결과 그래프
plt.figure(figsize=(20,10))
plt.plot(price)

plt.plot(np.arange(0, len(price),1), price, color='b')
plt.plot(np.arange(window_size, split_pt, 1), test_pred, color='g')
plt.plot(np.arange(split_pt, split_pt + len(train_pred), 1), train_pred, color='r')
plt.plot(np.arange(len(price), len(price) + len(after_pred), 1), after_pred, color='y')

print(
    '현재 종가: ', price[-1], #x는 마지막값 누락
    '예측 종가: ', test_pred[-1],
    '오 차 율: ',  (price[-1] - test_pred[-1]) / price[-1], sep='\n')

In [ ]:
trace = go.Scatter(x=np.arange(0,len(price),1), y=price, 
mode = 'lines', name='actual')
trace2 = go.Scatter(x=np.arange(window_size,split_pt,1), y=test_pred.reshape(len(test_pred)),
mode = 'lines', name='train')
trace3 = go.Scatter(x=np.arange(split_pt,split_pt + len(train_pred),1), y=train_pred.reshape(len(train_pred)), 
mode = 'lines', name='prediction')
trace4 = go.Scatter(x=np.arange(split_pt +len(train_pred), split_pt + len(train_pred) + len(after_pred), 1), y =after_pred.reshape(len(after_pred)),
mode = 'lines', name= 'after_pred')

data = [trace, trace2, trace3, trace4]
layout = go.Layout(title=Title)
fig = go.Figure(data=data, layout=layout)
py.offline.iplot(fig) #그래프 출력 
#py.offline.plot(fig, show_link = True, filename = Title + '.html') #다른이름으로 지정

from google.colab import files
import os

#os.rename('temp-plot.html', rename)
#files.download(filename=Title +'.html') #다운로드

In [ ]:
print('현재 종가: ', real_price[-1], 
      '\n다음 예측가격: \n', real_after_pred.astype(int))

In [ ]:
plt.plot(np.arange(0,len(after_pred), 1), after_pred, color='red')